In [39]:
### Save API Token, if needed
%set_env QXToken=5f375a49c4b5eb4c003b2bc33a2e96b3b4acb5acc84a44a21e8b793c87402bf1e900c28b4a09b084434b58633ebc764b020c7d2759699cd41242af3a8c8e313c


env: QXToken=5f375a49c4b5eb4c003b2bc33a2e96b3b4acb5acc84a44a21e8b793c87402bf1e900c28b4a09b084434b58633ebc764b020c7d2759699cd41242af3a8c8e313c


In [2]:
# import of required libraries and modules
from qc_grader.challenges.qgss_2024 import *

from math import pi
from qiskit.circuit.library import QFT
from qiskit.providers.fake_provider import GenericBackendV2, generic_backend_v2
generic_backend_v2._NOISE_DEFAULTS["cx"] = (5.99988e-06, 6.99988e-06, 1e-5, 5e-3)

from qiskit import transpile, QuantumCircuit
from qiskit.circuit import Gate
from qiskit.converters import circuit_to_dag
from qiskit.transpiler import CouplingMap, StagedPassManager, PassManager, AnalysisPass, TransformationPass
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit.transpiler.preset_passmanagers.common import generate_unroll_3q, generate_embed_passmanager
from qiskit.quantum_info import hellinger_fidelity
from qiskit.providers.basic_provider import BasicSimulator
from qiskit.dagcircuit import DAGCircuit
from qiskit_ibm_runtime.fake_provider import FakeTorino

# Transpiler Passes
## Layout passes
from qiskit.transpiler.passes.layout.csp_layout import CSPLayout
from qiskit.transpiler.passes.layout.dense_layout import DenseLayout
from qiskit.transpiler.passes.layout.sabre_layout import SabreLayout
from qiskit.transpiler.passes.layout.vf2_layout import VF2Layout
from qiskit.transpiler.passes.layout.trivial_layout import TrivialLayout

## Routing passes
from qiskit.transpiler.passes.routing.basic_swap import BasicSwap
from qiskit.transpiler.passes.routing.lookahead_swap import LookaheadSwap
from qiskit.transpiler.passes.routing.sabre_swap import SabreSwap
from qiskit.transpiler.passes.routing.stochastic_swap import StochasticSwap
from qiskit.transpiler.passes.routing.star_prerouting import StarPreRouting

## Synthesis passes (passes for the translation stage)
from qiskit.circuit import SessionEquivalenceLibrary
from qiskit.circuit.equivalence_library import SessionEquivalenceLibrary
from qiskit.transpiler.passes.basis.basis_translator import BasisTranslator
from qiskit.transpiler.passes.synthesis.high_level_synthesis import HighLevelSynthesis
### The next pass could also be considered an optimization pass.
from qiskit.transpiler.passes.synthesis.unitary_synthesis import UnitarySynthesis

## Optimization passes
from qiskit.transpiler.passes.optimization.collect_1q_runs import Collect1qRuns
from qiskit.transpiler.passes.optimization.collect_2q_blocks import Collect2qBlocks
from qiskit.transpiler.passes.optimization.consolidate_blocks import ConsolidateBlocks
from qiskit.transpiler.passes.optimization.commutative_cancellation import CommutativeCancellation

In [16]:
num_qubits = 10
qc = QFT(num_qubits, do_swaps=False)
qc.draw()

┌──────┐
q_0: ┤0     ├
     │      │
q_1: ┤1     ├
     │      │
q_2: ┤2     ├
     │      │
q_3: ┤3     ├
     │      │
q_4: ┤4     ├
     │  QFT │
q_5: ┤5     ├
     │      │
q_6: ┤6     ├
     │      │
q_7: ┤7     ├
     │      │
q_8: ┤8     ├
     │      │
q_9: ┤9     ├
     └──────┘

In [17]:
def get_qc_characteristics(qc):
    depth = qc.depth()   
    num_qubits =qc.num_qubits
    ops = qc.count_ops()
    num_multi_qubit_ops = sum(1 for op in qc.data if len(op[1]) > 1)
    return {"depth":depth, "num_qubits":num_qubits, "ops":ops, "num_multi_qubit_ops": num_multi_qubit_ops}    

In [9]:
#grade_lab1_ex1(get_qc_characteristics)

Submitting your answer. Please wait...
Congratulations 🎉! Your answer is correct and has been submitted.


In [16]:
qc = QuantumCircuit(8)
qc.h(0)
qc.cx(0, 1)
qc.cx(1, 2)
qc.h(4)
qc.draw()

┌───┐          
q_0: ┤ H ├──■───────
     └───┘┌─┴─┐     
q_1: ─────┤ X ├──■──
          └───┘┌─┴─┐
q_2: ──────────┤ X ├
               └───┘
q_3: ───────────────
     ┌───┐          
q_4: ┤ H ├──────────
     └───┘          
q_5: ───────────────
                    
q_6: ───────────────
                    
q_7: ───────────────

In [7]:
get_qc_characteristics(qc)

{'depth': 1,
 'num_qubits': 10,
 'ops': OrderedDict([('QFT', 1)]),
 'num_multi_qubit_ops': 1}

In [18]:
# print quantum circuit characteristics
def print_qc_characteristics(qc):
    characteristics = get_qc_characteristics(qc)
    print("Quantum circuit characteristics")
    print("  Depth:", characteristics['depth'])
    print("  Number of qubits:", characteristics['num_qubits'])
    print("  Operations:", dict(characteristics['ops']))
    print("  Number of multi-qubit Operations:", characteristics['num_multi_qubit_ops'])
    
print_qc_characteristics(qc)

Quantum circuit characteristics
  Depth: 1
  Number of qubits: 10
  Operations: {'QFT': 1}
  Number of multi-qubit Operations: 1


In [19]:
num_qubits = 10
qc = QFT(num_qubits, do_swaps=False)
qc.draw()

┌──────┐
q_0: ┤0     ├
     │      │
q_1: ┤1     ├
     │      │
q_2: ┤2     ├
     │      │
q_3: ┤3     ├
     │      │
q_4: ┤4     ├
     │  QFT │
q_5: ┤5     ├
     │      │
q_6: ┤6     ├
     │      │
q_7: ┤7     ├
     │      │
q_8: ┤8     ├
     │      │
q_9: ┤9     ├
     └──────┘

In [6]:
qc_dec = qc.decompose()
get_qc_characteristics(qc_dec)
qc_dec.draw(fold=-1)

┌───┐
q_0: ────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────■────────────────────────────────────────────■──────────────────────────────────────■──────────────────────────────────■───────────────────────────■───────────────────────■──────────────────■─────────────■────────■───────┤ H ├
                                                                                                                                                                                                                                                                     │                                            │                                      │                                  │                           │                       │                  │       ┌───┐ │        │P(π/2) └───┘
q_1: ──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────■──────────────────────────────────────■──────────┼──────────────────────────────────■─────────┼────────────────────────────■─────────┼────────────────────────■─────────┼──────────────────■────────┼──────────────■────────┼─────────■────────┼───────┤ H ├─┼────────■────────────
                                                                                                                                                                                                                   │                                      │          │                                  │         │                            │         │                        │         │                  │        │        ┌───┐ │        │         │P(π/2)  │       └───┘ │P(π/4)               
q_2: ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────■──────────────────────────────────■─────────┼────────────────────────────■─────────┼──────────┼────────────────────────■─────────┼─────────┼───────────────────■────────┼─────────┼───────────────■────────┼─────────┼─────────■────────┼────────┼────────┤ H ├─┼────────┼─────────■────────┼─────────────■─────────────────────
                                                                                                                                                                      │                                  │         │                            │         │          │                        │         │         │                   │        │         │         ┌───┐ │        │         │         │P(π/2)  │        │        └───┘ │P(π/4)  │                  │P(π/8)                             
q_3: ───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────■───────────────────────────■─────────┼────────────────────────■─────────┼─────────┼───────────────────■────────┼─────────┼──────────┼───────────────■────────┼─────────┼─────────┼──────────■────────┼────────┼─────────┼─────────┤ H ├─┼────────┼─────────┼─────────■────────┼────────┼──────────────■────────┼──────────────────■───────────────────────────────────
                                                                                                                                │                           │         │                        │         │         │                   │        │         │          │         ┌───┐ │        │         │         │          │P(π/2)  │        │         │         └───┘ │P(π/4)  │         │                  │P(π/8)  │                       │P(π/16)                                               
q_4: ─────────────────────────────────────────────────────────────────────────────────────

In [20]:
backend = GenericBackendV2(num_qubits)
print("Supported basis gates:", backend.operation_names)

Supported basis gates: ['cx', 'id', 'rz', 'sx', 'x', 'reset', 'delay', 'measure']


In [21]:
qc_synth = generate_preset_pass_manager(2, backend=backend).run(qc)
qc_synth.draw(fold=-1)

global phase: 0
                                                                                                                                                            ┌───┐┌───────────┐              ┌───┐┌──────────┐    ┌───┐    ┌───────────┐   ┌───┐    ┌──────────┐                        ┌───┐┌──────────┐                      ┌───┐     ┌─────────┐    ┌───┐     ┌──────────┐   ┌───┐    ┌──────────┐   ┌────┐   ┌────────────┐                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    
q_5 -> 0 ───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┤ X ├┤ Rz(-π/32) ├──────────────┤ X ├┤ Rz(π/32) ├────┤ X ├────┤ Rz(-π/16) ├───┤ X ├────┤ Rz(π/16) ├────────────────────────┤ X ├┤ Rz(-π/8) ├──────────────────────┤ X ├─────┤ Rz(π/8) ├────┤ X ├─────┤ Rz(-π/4) ├───┤ X ├────┤ Rz(3π/4) ├───┤ √X ├───┤ Rz(3.0925) ├──────────────────────────────────────────────────────────────────────────────────────────────────■─────────────────────────────────────────────────■────────────────────■─────────────────────────────────────■─────────────────────────────────■────────────────────────────────────────────────────■──────────────────────────■───────────────────────────────────────■──────────────────■───────────────────────────────────■───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
                                        ┌───┐┌──────────┐┌───┐┌──────────┐   ┌────┐   ┌────────────┐                                                        └─┬─┘└───────────┘              └─┬─┘└──────────┘    └─┬─┘    └───────────┘   └─┬─┘    └──────────┘                        └─┬─┘└──────────┘                      └─┬─┘     └─────────┘    └─┬─┘     └──────────┘   └─┬─┘    └──────────┘   └────┘   └────────────┘                                                                                                  │                                                 │                    │                                     │                                 │                                                    │                          │                                       │                  │                                   │                                                                                                                                                                                                                   
q_8 -> 1 ───────────────────────────────┤ X ├┤ Rz(-π/4) ├┤ X ├┤ Rz(3π/4) ├───┤ √X ├───┤ Rz(3.1355) ├──────────────────■──────────────────────■────────────────┼────────■──────────────────────┼───────■────────────■────────────────────────■─────────────────────────■──────────────────┼────────────────■─────────────────────┼────────────■───────────┼────────────■───────────┼───────────────────────■─────────────────────────────────────────────■──────────────────────────■────────────────────────────────────────■────┼──────────────────────────────■──────────────────┼────────────────────┼─────────■───────────────────────────┼─────────────────────────────────┼────────────────────────────────────────────────────┼──────────────────────────┼───────────────────────────────────────┼──────────────────┼───────────────────────────────────┼────────────────────

In [10]:
print_qc_characteristics(qc_synth)

Quantum circuit characteristics
  Depth: 65
  Number of qubits: 10
  Operations: {'rz': 101, 'cx': 90, 'sx': 10}
  Number of multi-qubit Operations: 90


In [11]:
qc = QFT(num_qubits=10)
qc.draw()

┌──────┐
q_0: ┤0     ├
     │      │
q_1: ┤1     ├
     │      │
q_2: ┤2     ├
     │      │
q_3: ┤3     ├
     │      │
q_4: ┤4     ├
     │  QFT │
q_5: ┤5     ├
     │      │
q_6: ┤6     ├
     │      │
q_7: ┤7     ├
     │      │
q_8: ┤8     ├
     │      │
q_9: ┤9     ├
     └──────┘

In [16]:



cm = [[0, 1], [1, 2], [2, 3], [3, 4], [4, 5], [5, 6], [6, 7], [7, 8], [8, 9]]

qc_routed = transpile(qc, backend=backend, coupling_map=cm)

In [44]:
# grade your work!
grade_lab1_ex2(qc_routed)

Submitting your answer. Please wait...
Congratulations 🎉! Your answer is correct and has been submitted.


In [20]:
#qc_routed.draw()

In [22]:

cm = CouplingMap([[i, i+1] for i in range(9)])


pm_staged = StagedPassManager()
# replace the n-qubit QFT operation with its decomposition in two-qubit gates
pm_staged.init = generate_unroll_3q(None)
# initialize the layout stage with an empty pass manager
pm_staged.layout = PassManager()
# set a 'trivial' initial layout, i.e. each qubit in the quantum circuit with index i 
# is mapped to the physical qubit on a device with the same index
pm_staged.layout += TrivialLayout(cm)

# do not modify the next line
pm_staged.layout += generate_embed_passmanager(cm)

In [23]:
# See the first cells in this notebook or
#  https://github.com/Qiskit/qiskit/tree/main/qiskit/transpiler/passes/routing for potential routing passes
# pm_staged.routing = PassManager(MyPass)

#pm_staged.routing = PassManager(BasicSwap(cm))
pm_staged.routing = PassManager(StochasticSwap(cm))
#pm_staged.routing = PassManager(LookaheadSwap(cm))


qc = QFT(num_qubits=10)
qc_decomposed = qc.decompose()

# Run the staged pass manager on the decomposed circuit
qc_routed = pm_staged.run(qc_decomposed)




#qc_routed = pm_staged.run(qc_decomposed)
print_qc_characteristics(qc_routed)
qc_routed.draw(fold=-1)

Quantum circuit characteristics
  Depth: 41
  Number of qubits: 10
  Operations: {'swap': 70, 'cp': 45, 'h': 10}
  Number of multi-qubit Operations: 115


q_0 -> 0 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────■──────────X────────────────────────────────────────────────────────────────────────────────────────────────X───────────────────────────
                                                                                                                      │P(π/512)  │                                                                                                │                           
q_1 -> 1 ───────────────────────────────────────────────────────────────────────────────────────────────■──────────X──■──────────X──■──────────X────────────────────────────────────────────────────────────────────────────X─────X─────────X─────────────────
                                                                                                        │P(π/256)  │                │P(π/256)  │                                                                            │               │                 
q_2 -> 2 ─────────────────────────────────────────────────────────────────────────────────■──────────X──■──────────X──■──────────X──■──────────X──■──────────X────────────────────────────────────────────────────────X─────X───────────X───X──────X──────────
                                                                                          │P(π/128)  │                │P(π/128)  │                │P(π/128)  │                                                        │                 │          │          
q_3 -> 3 ────────────────────────────────────────────────────────────────────■─────────X──■──────────X───■─────────X──■──────────X───■─────────X──■──────────X──■─────────X─────────────────────────────────────X─────X───────────X─────X───────X──X─────X────
                                                                             │P(π/64)  │                 │P(π/64)  │                 │P(π/64)  │                │P(π/64)  │                                     │                 │             │        │    
q_4 -> 4 ───────────────────────────────────────────────────────■─────────X──■─────────X───■─────────X───■─────────X───■─────────X───■─────────X───■─────────X──■─────────X──■─────────X──────────────────X─────X───────────X─────X─────────X───X─────X──X──X─
                                                                │P(π/32)  │                │P(π/32)  │                 │P(π/32)  │                 │P(π/32)  │               │P(π/32)  │                  │                 │               │         │     │ 
q_5 -> 5 ──────────────────────────────────────────■─────────X──■─────────X──■─────────X───■─────────X───■─────────X───■─────────X───■─────────X───■─────────X──■─────────X──■─────────X──■─────────X─────X───────────X─────X───────────X───X──────X──X──X──X─
                                                   │P(π/16)  │               │P(π/16)  │                 │P(π/16)  │                 │P(π/16)  │                │P(π/16)  │               │P(π/16)  │                 │                 │          │     │    
q_6 -> 6 ──────────────────────────────■────────X──■─────────X──■─────────X──■─────────X───■─────────X───■─────────X───■─────────X───■─────────X───■─────────X──■─────────X──■─────────X──■─────────X──■────────X─────X───────────X─────X───────X──X──X──X────
                                       │P(π/8)  │               │P(π/8)   │                │P(π/8)   │                 │P(π/8)   │                 │P(π/8)   │               │P(π/8)   │               │P(π/8)  │                 │             │     │       
q_7 -> 7 ──────────────────■────────X──■────────X──■─────────X──■─────────X──■─────────X───■─────────X───■─────────X───■─────────X───■─────────X───■─────────X──■─────────X──■─────────X──■─────────X──■────────X──■────────X─────X─────────X───X──X──X───────
                           │P(π/4)  │              │P(π/4)   │               │P(π/4)   │                 │P(π/4)   │                 │P(π/4)   │                │P(π/4)   │    

In [10]:
# grade your work!
grade_lab1_ex3(pm_staged)

Submitting your answer. Please wait...
Congratulations 🎉! Your answer is correct and has been submitted.


In [15]:
print_qc_characteristics(qc_decomposed)

Quantum circuit characteristics
  Depth: 20
  Number of qubits: 10
  Operations: {'cp': 45, 'h': 10, 'swap': 5}
  Number of multi-qubit Operations: 50


In [10]:
backend = GenericBackendV2(num_qubits)
print("Supported basis gates:", backend.operation_names)

Supported basis gates: ['cx', 'id', 'rz', 'sx', 'x', 'reset', 'delay', 'measure']


In [24]:
basis_gates = ['cx', 'id', 'rz', 'sx', 'x', 'reset', 'delay', 'measure']

In [25]:
# See the first cells in this notebook or
#   https://github.com/Qiskit/qiskit/tree/main/qiskit/transpiler/passes/synthesis for potential translation passes

# Define the basis gates set for the backend
basis_gates = ['cx', 'id', 'rz', 'sx', 'x', 'reset', 'delay', 'measure'] # Adjust this list based on the actual backend's basis gates

# Add a translation pass to ensure the circuit uses only the specified basis gates
pm_staged.translation = PassManager([BasisTranslator(SessionEquivalenceLibrary, basis_gates)])



qc_routed_synth = pm_staged.run(qc)
print_qc_characteristics(qc_routed_synth)
qc_routed_synth.draw(fold=-1)

Quantum circuit characteristics
  Depth: 171
  Number of qubits: 10
  Operations: {'cx': 342, 'rz': 155, 'sx': 10}
  Number of multi-qubit Operations: 342


global phase: 1.5723
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                ┌───┐         ┌───────────┐                                                                                     ┌───────────┐                                                                                                                                                  ┌───┐                                                                                                                                                                                                                                                                                                                                                                                                                                                                           ┌───┐                                                                                                                                                       
q_0 -> 0 ────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────■──────────┤ X ├──────■──┤ Rz(π/256) ├────────────────────────────────────────────────■────────────────────────────■───────┤ Rz(π/512) ├─────────────────────────────────────────────────■─────────────────────────■───────────────────────────────────────────────────────────────■──────┤ X ├─────■──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────■──┤ X ├──■────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             ┌───┐               ┌─

In [35]:
# grade your work!
grade_lab1_ex4(pm_staged)

Submitting your answer. Please wait...
Congratulations 🎉! Your answer is correct and has been submitted.


In [26]:
qk_qc = generate_preset_pass_manager(2, backend=backend).run(qc)
print_qc_characteristics(qk_qc)

Quantum circuit characteristics
  Depth: 65
  Number of qubits: 10
  Operations: {'rz': 101, 'cx': 90, 'sx': 10}
  Number of multi-qubit Operations: 90


In [27]:
def noisy_sim(qc, backend):
    # We add measurement operations to the input quantum circuit and then run it on the specified backend
    # A GenericBackendV2 automatically constructs a default model of the expected noise processes, 
    # so backend.run would return noisy simulation results    
    return backend.run(qc.measure_all(inplace=False), shots=7*1024).result().get_counts()

own_transpiler_sim = noisy_sim(qc_routed_synth, backend)
qiskit_transpiler_sim = noisy_sim(qk_qc, backend)
reference_sim =  noisy_sim(transpile(qc.decompose(), backend=backend), BasicSimulator())

print("Own transpiler fidelity", round(hellinger_fidelity(own_transpiler_sim, reference_sim), 4))
print("Qiskit transpiler fidelity", round(hellinger_fidelity(qiskit_transpiler_sim, reference_sim), 4))

Own transpiler fidelity 0.1903
Qiskit transpiler fidelity 0.5702


In [28]:
pm_opt = StagedPassManager()

pm_opt.init = generate_unroll_3q(None)

pm_opt.layout = PassManager()
pm_opt.layout += TrivialLayout(cm)

pm_opt.routing = PassManager(StarPreRouting())

pm_opt.translation =  PassManager(BasisTranslator(SessionEquivalenceLibrary,backend.operation_names))
pm_opt.translation += HighLevelSynthesis()

pm_opt.optimization = PassManager()
pm_opt.optimization += Collect2qBlocks()
pm_opt.optimization += ConsolidateBlocks()
pm_opt.optimization += UnitarySynthesis(basis_gates = backend.operation_names)
pm_opt.optimization += CommutativeCancellation()


pm_opt.layout += generate_embed_passmanager(cm)
qc_opt = pm_opt.run(qc)

In [41]:
# grade your work!
grade_lab1_ex5(pm_opt)

Submitting your answer. Please wait...
Congratulations 🎉! Your answer is correct and has been submitted.


In [29]:
print_qc_characteristics(qc_opt)
reduction_ratio = round(100-100*(get_qc_characteristics(qc_opt)['num_multi_qubit_ops']/get_qc_characteristics(qk_qc)['num_multi_qubit_ops']), 3)
print("Reduction in two-qubit gates compared to qiskit {}%!".format(reduction_ratio))
qc_opt.draw(fold=-1)

Quantum circuit characteristics
  Depth: 170
  Number of qubits: 10
  Operations: {'rz': 322, 'sx': 186, 'cx': 137, 'x': 76}
  Number of multi-qubit Operations: 137
Reduction in two-qubit gates compared to qiskit -52.222%!


┌──────────┐┌────┐                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             ┌───┐     ┌──────────┐               ┌───┐    ┌────────────┐    ┌───┐       ┌─────────┐                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                ┌───┐     ┌───┐
q_0 -> 0 ─┤ Rz(-π/4) ├┤ √X ├─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┤ X ├─────┤ Rz(-π/2) ├───────────────┤ X ├────┤ Rz(1.5677) ├────┤ X ├───────┤ Rz(π/4) ├──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────

In [30]:
opt_transpiler_sim = noisy_sim(qc_opt, backend)

print("Own transpiler fidelity", round(hellinger_fidelity(own_transpiler_sim, reference_sim), 4))
print("Qiskit transpiler fidelity", round(hellinger_fidelity(qiskit_transpiler_sim, reference_sim), 4))
print("Own optimized transpiler fidelity", round(hellinger_fidelity(opt_transpiler_sim, reference_sim), 4))

Own transpiler fidelity 0.1903
Qiskit transpiler fidelity 0.5702
Own optimized transpiler fidelity 0.8981


In [31]:
from qiskit.transpiler.basepasses import AnalysisPass
from qiskit.circuit import Qubit
from qiskit import QuantumCircuit
from qiskit.circuit.library import QFT
from qiskit.transpiler import PassManager, CouplingMap

class GatesPerQubit(AnalysisPass):
    def __init__(self):
        super().__init__()

    def run(self, dag):
        one_q_op = {qubit: 0 for qubit in dag.qubits}
        two_q_op = {qubit: 0 for qubit in dag.qubits}

        for node in dag.topological_op_nodes():
            if node.op.num_qubits == 1:
                for qubit in node.qargs:
                    one_q_op[qubit] += 1
            elif node.op.num_qubits == 2:
                for qubit in node.qargs:
                    two_q_op[qubit] += 1

        self.property_set["one_q_op"] = one_q_op
        self.property_set["two_q_op"] = two_q_op

# Example usage
qc = QFT(num_qubits=10)
qc_decomposed = qc.decompose()

# Create the custom pass
count_gates_pass = GatesPerQubit()

# Add it to a pass manager
pm = PassManager(count_gates_pass)

# Run the pass manager on the circuit
pm.run(qc_decomposed)

# Retrieve and print the results
one_q_op = count_gates_pass.property_set['one_q_op']
two_q_op = count_gates_pass.property_set['two_q_op']

print("Single-qubit gate counts per qubit:", one_q_op)
print("Two-qubit gate counts per qubit:", two_q_op)


Single-qubit gate counts per qubit: {Qubit(QuantumRegister(10, 'q'), 0): 1, Qubit(QuantumRegister(10, 'q'), 1): 1, Qubit(QuantumRegister(10, 'q'), 2): 1, Qubit(QuantumRegister(10, 'q'), 3): 1, Qubit(QuantumRegister(10, 'q'), 4): 1, Qubit(QuantumRegister(10, 'q'), 5): 1, Qubit(QuantumRegister(10, 'q'), 6): 1, Qubit(QuantumRegister(10, 'q'), 7): 1, Qubit(QuantumRegister(10, 'q'), 8): 1, Qubit(QuantumRegister(10, 'q'), 9): 1}
Two-qubit gate counts per qubit: {Qubit(QuantumRegister(10, 'q'), 0): 10, Qubit(QuantumRegister(10, 'q'), 1): 10, Qubit(QuantumRegister(10, 'q'), 2): 10, Qubit(QuantumRegister(10, 'q'), 3): 10, Qubit(QuantumRegister(10, 'q'), 4): 10, Qubit(QuantumRegister(10, 'q'), 5): 10, Qubit(QuantumRegister(10, 'q'), 6): 10, Qubit(QuantumRegister(10, 'q'), 7): 10, Qubit(QuantumRegister(10, 'q'), 8): 10, Qubit(QuantumRegister(10, 'q'), 9): 10}


In [20]:
# grade your work!
grade_lab1_ex6(GatesPerQubit)

Submitting your answer. Please wait...
Congratulations 🎉! Your answer is correct and has been submitted.


In [32]:
qc = QFT(4, do_swaps=False)
qc = generate_preset_pass_manager(2, backend=backend).run(qc)
gpq = GatesPerQubit()
gpq(qc)
print("single-qubit gates on qubit", gpq.property_set["one_q_op"])
print("two-qubit gates on qubit", gpq.property_set["two_q_op"])
qc.draw(fold=-1, idle_wires=False)

single-qubit gates on qubit {Qubit(QuantumRegister(10, 'q'), 0): 4, Qubit(QuantumRegister(10, 'q'), 1): 0, Qubit(QuantumRegister(10, 'q'), 2): 0, Qubit(QuantumRegister(10, 'q'), 3): 3, Qubit(QuantumRegister(10, 'q'), 4): 0, Qubit(QuantumRegister(10, 'q'), 5): 0, Qubit(QuantumRegister(10, 'q'), 6): 0, Qubit(QuantumRegister(10, 'q'), 7): 8, Qubit(QuantumRegister(10, 'q'), 8): 0, Qubit(QuantumRegister(10, 'q'), 9): 6}
two-qubit gates on qubit {Qubit(QuantumRegister(10, 'q'), 0): 6, Qubit(QuantumRegister(10, 'q'), 1): 0, Qubit(QuantumRegister(10, 'q'), 2): 0, Qubit(QuantumRegister(10, 'q'), 3): 6, Qubit(QuantumRegister(10, 'q'), 4): 0, Qubit(QuantumRegister(10, 'q'), 5): 0, Qubit(QuantumRegister(10, 'q'), 6): 0, Qubit(QuantumRegister(10, 'q'), 7): 6, Qubit(QuantumRegister(10, 'q'), 8): 0, Qubit(QuantumRegister(10, 'q'), 9): 6}


global phase: 0
                                        ┌───┐┌──────────┐┌───┐┌──────────┐   ┌────┐   ┌──────────┐                                                                                                                                                            
q_2 -> 0 ───────────────────────────────┤ X ├┤ Rz(-π/4) ├┤ X ├┤ Rz(3π/4) ├───┤ √X ├───┤ Rz(7π/8) ├──────────────────────────■────────────────────────────■───────■───────────────────────■────────────────────────────────────────────────────────────────────
         ┌─────────┐┌────┐┌────────────┐└─┬─┘└──────────┘└─┬─┘└──────────┘   └────┘   └──────────┘                          │                            │       │                       │                                                                    
q_3 -> 3 ┤ Rz(π/2) ├┤ √X ├┤ Rz(15π/16) ├──■────────────────■───────■───────────────────────■───────────■────────────────────┼───────■────────────────────┼───────┼───────────────────────┼────────────────────────────────────────────────────────────────────
         └─────────┘└────┘└────────────┘                           │                       │         ┌─┴─┐   ┌───────────┐  │     ┌─┴─┐    ┌──────────┐  │     ┌─┴─┐    ┌──────────┐   ┌─┴─┐    ┌─────────┐┌───┐┌──────────┐┌───┐┌──────────┐┌────┐┌─────────┐
q_0 -> 7 ──────────────────────────────────────────────────────────┼───────────────────────┼─────────┤ X ├───┤ Rz(-π/16) ├──┼─────┤ X ├────┤ Rz(π/16) ├──┼─────┤ X ├────┤ Rz(-π/8) ├───┤ X ├────┤ Rz(π/8) ├┤ X ├┤ Rz(-π/4) ├┤ X ├┤ Rz(3π/4) ├┤ √X ├┤ Rz(π/2) ├
                                                                 ┌─┴─┐    ┌──────────┐   ┌─┴─┐    ┌──┴───┴──┐└───────────┘┌─┴─┐┌──┴───┴───┐└──────────┘┌─┴─┐┌──┴───┴───┐└──┬────┬──┘┌──┴───┴───┐└─────────┘└─┬─┘└──────────┘└─┬─┘└──────────┘└────┘└─────────┘
q_1 -> 9 ────────────────────────────────────────────────────────┤ X ├────┤ Rz(-π/8) ├───┤ X ├────┤ Rz(π/8) ├─────────────┤ X ├┤ Rz(-π/4) ├────────────┤ X ├┤ Rz(3π/4) ├───┤ √X ├───┤ Rz(3π/4) ├─────────────■────────────────■───────────────────────────────
                                                                 └───┘    └──────────┘   └───┘    └─────────┘             └───┘└──────────┘            └───┘└──────────┘   └────┘   └──────────┘

In [33]:
pg = Gate('Peres', 3, params=[], label='PG')

In [34]:
qc_pg = QuantumCircuit(3)
qc_pg.append(pg, [0, 1, 2])
qc_pg.draw()

┌─────┐
q_0: ┤0    ├
     │     │
q_1: ┤1 PG ├
     │     │
q_2: ┤2    ├
     └─────┘

In [35]:
def get_qc_in(nq):
    # QFT circuit, feel free to use a previously defined pass manager for the QFT circuit
    qc_qft = QFT(nq, do_swaps=False)
    # part of the circuit including the Peres gate
    qc_inner = QuantumCircuit(nq)
    for i in range(1, nq - 1):
        qc_inner.append(pg, [nq - i - 2, nq - i - 1, nq - 1])

    qc_in = QuantumCircuit(nq)
    # add QFT circuit to qc_in
    qc_in.compose(qc_qft, range(nq), inplace=True)

    # perform swap gates
    for i in range(nq // 2):
        qc_in.cx(i, nq - i - 1)
        qc_in.cx(nq - i - 1, i)
        qc_in.cx(i, nq - i - 1)

    qc_in.rz(pi, nq - 1)
    # add circuit with peres gates
    qc_in.compose(qc_inner, range(nq), inplace=True)

    # perform swap gates
    for i in range(nq // 2):
        qc_in.cx(i, nq - i - 1)
        qc_in.cx(nq - i - 1, i)
        qc_in.cx(i, nq - i - 1)
    # add inverse QFT circuit
    qc_in.compose(qc_qft.inverse(), range(nq), inplace=True)
    return qc_in

nq = 5
qc_in = get_qc_in(nq)
qc_in.draw(fold=-1)

┌──────┐          ┌───┐                                 ┌─────┐          ┌───┐               ┌───────┐
q_0: ┤0     ├──■───────┤ X ├───────■─────────────────────────┤0    ├──■───────┤ X ├───────■───────┤0      ├
     │      │  │       └─┬─┘┌───┐  │                  ┌─────┐│     │  │       └─┬─┘┌───┐  │       │       │
q_1: ┤1     ├──┼────■────┼──┤ X ├──┼──────■───────────┤0    ├┤1    ├──┼────■────┼──┤ X ├──┼────■──┤1      ├
     │      │  │    │    │  └─┬─┘  │      │    ┌─────┐│     ││     │  │    │    │  └─┬─┘  │    │  │       │
q_2: ┤2 QFT ├──┼────┼────┼────┼────┼──────┼────┤0    ├┤1    ├┤  PG ├──┼────┼────┼────┼────┼────┼──┤2 IQFT ├
     │      │  │  ┌─┴─┐  │    │    │    ┌─┴─┐  │     ││  PG ││     │  │  ┌─┴─┐  │    │    │  ┌─┴─┐│       │
q_3: ┤3     ├──┼──┤ X ├──┼────■────┼────┤ X ├──┤1 PG ├┤     ├┤     ├──┼──┤ X ├──┼────■────┼──┤ X ├┤3      ├
     │      │┌─┴─┐└───┘  │       ┌─┴─┐┌─┴───┴─┐│     ││     ││     │┌─┴─┐└───┘  │       ┌─┴─┐└───┘│       │
q_4: ┤4     ├┤ X ├───────■───────┤ X ├┤ Rz(π) ├┤2    ├┤2    ├┤2    ├┤ X ├───────■───────┤ X ├─────┤4      ├
     └──────┘└───┘               └───┘└───────┘└─────┘└─────┘└─────┘└───┘               └───┘     └───────┘

In [36]:
from qiskit.transpiler.basepasses import TransformationPass
from qiskit.dagcircuit import DAGCircuit
from qiskit.circuit import Gate, QuantumCircuit
from math import pi

# Define the Peres gate
pg = Gate('Peres', 3, params=[], label='PG')

class PeresGateTranslation(TransformationPass):
    def __init__(self):
        super().__init__()

    def get_peres_decomposition(self):
        qcsx = QuantumCircuit(2)
        qcsx.rz(pi / 4, 0)
        qcsx.rz(pi / 2, 1)
        qcsx.sx(1)
        qcsx.rz(pi / 2, 1)
        qcsx.cx(0, 1)
        qcsx.rz(-pi / 4, 1)
        qcsx.cx(0, 1)
        qcsx.rz(3 * pi / 4, 1)
        qcsx.sx(1)
        qcsx.rz(pi / 2, 1)

        qcsx_inv = QuantumCircuit(2)
        qcsx_inv.rz(pi / 4, 1)
        qcsx_inv.cx(0, 1)
        qcsx_inv.rz(-pi / 4, 1)
        qcsx_inv.cx(0, 1)
        qcsx_inv.rz(pi / 2, 0)
        qcsx_inv.rz(pi / 2, 1)
        qcsx_inv.cx(0, 1)
        qcsx_inv.rz(pi / 2, 1)
        qcsx_inv.sx(1)
        qcsx_inv.rz(-3 * pi / 4, 1)
        qcsx_inv.sx(1)
        qcsx_inv.cx(0, 1)
        qcsx_inv.sx(1)
        qcsx_inv.rz(-3 * pi / 4, 1)
        qcsx_inv.sx(1)
        qcsx_inv.rz(-3 * pi / 4, 1)
        qcsx_inv.cx(0, 1)
        qcsx_inv.rz(-pi / 4, 1)
        qcsx_inv.cx(0, 1)
        qcsx_inv.rz(pi / 4, 0)

        qc_dec = QuantumCircuit(3)
        qc_dec.cx(0, 1)
        qc_dec.cx(1, 0)
        qc_dec.cx(0, 1)
        qc_dec.compose(qcsx, [1, 2], inplace=True)
        qc_dec.cx(0, 1)
        qc_dec.cx(1, 0)
        qc_dec.cx(0, 1)
        qc_dec.compose(qcsx, [1, 2], inplace=True)
        qc_dec.cx(0, 1)
        qc_dec.compose(qcsx_inv, [1, 2], inplace=True)
        qc_dec.cx(0, 1)
        qc_dec.cx(0, 1)
        return qc_dec

    def run(self, dag: DAGCircuit):
        for node in dag.named_nodes('Peres'):
            decomposition_dag = circuit_to_dag(self.get_peres_decomposition())
            dag.substitute_node_with_dag(node, decomposition_dag)
        return dag


In [37]:
from qiskit.circuit.library import QFT

def get_qc_in(nq):
    qc_qft = QFT(nq, do_swaps=False)
    qc_inner = QuantumCircuit(nq)
    for i in range(1, nq - 1):
        qc_inner.append(pg, [nq - i - 2, nq - i - 1, nq - 1])

    qc_in = QuantumCircuit(nq)
    qc_in.compose(qc_qft, range(nq), inplace=True)
    for i in range(nq // 2):
        qc_in.cx(i, nq - i - 1)
        qc_in.cx(nq - i - 1, i)
        qc_in.cx(i, nq - i - 1)
    qc_in.rz(pi, nq - 1)
    qc_in.compose(qc_inner, range(nq), inplace=True)
    for i in range(nq // 2):
        qc_in.cx(i, nq - i - 1)
        qc_in.cx(nq - i - 1, i)
        qc_in.cx(i, nq - i - 1)
    qc_in.compose(qc_qft.inverse(), range(nq), inplace=True)
    return qc_in

nq = 5
qc_in = get_qc_in(nq)
qc_in.draw(fold=-1)


┌──────┐          ┌───┐                                 ┌─────┐          ┌───┐               ┌───────┐
q_0: ┤0     ├──■───────┤ X ├───────■─────────────────────────┤0    ├──■───────┤ X ├───────■───────┤0      ├
     │      │  │       └─┬─┘┌───┐  │                  ┌─────┐│     │  │       └─┬─┘┌───┐  │       │       │
q_1: ┤1     ├──┼────■────┼──┤ X ├──┼──────■───────────┤0    ├┤1    ├──┼────■────┼──┤ X ├──┼────■──┤1      ├
     │      │  │    │    │  └─┬─┘  │      │    ┌─────┐│     ││     │  │    │    │  └─┬─┘  │    │  │       │
q_2: ┤2 QFT ├──┼────┼────┼────┼────┼──────┼────┤0    ├┤1    ├┤  PG ├──┼────┼────┼────┼────┼────┼──┤2 IQFT ├
     │      │  │  ┌─┴─┐  │    │    │    ┌─┴─┐  │     ││  PG ││     │  │  ┌─┴─┐  │    │    │  ┌─┴─┐│       │
q_3: ┤3     ├──┼──┤ X ├──┼────■────┼────┤ X ├──┤1 PG ├┤     ├┤     ├──┼──┤ X ├──┼────■────┼──┤ X ├┤3      ├
     │      │┌─┴─┐└───┘  │       ┌─┴─┐┌─┴───┴─┐│     ││     ││     │┌─┴─┐└───┘  │       ┌─┴─┐└───┘│       │
q_4: ┤4     ├┤ X ├───────■───────┤ X ├┤ Rz(π) ├┤2    ├┤2    ├┤2    ├┤ X ├───────■───────┤ X ├─────┤4      ├
     └──────┘└───┘               └───┘└───────┘└─────┘└─────┘└─────┘└───┘               └───┘     └───────┘

In [38]:
from qiskit.transpiler import PassManager
from qiskit.converters import circuit_to_dag

# Create and run the pass manager with the custom transformation pass
pm = PassManager()
pm.append(PeresGateTranslation())
qc_transformed = pm.run(qc_in)

# Draw the transformed circuit
qc_transformed.draw(fold=-1)


┌──────┐               ┌───┐                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      ┌───┐                                                  ┌───┐                                                                                                                                                                                                                                                                                     ┌───┐          ┌───────┐
q_0: ┤0     ├──■────────────┤ X ├───────■───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────■──┤ X ├──■────────────────────────────────────────■──────┤ X ├──────■────────────────────────────────────────────────────────────■──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────■────■────■────────────┤ X ├───────■──┤0      ├
     │      │  │       ┌───┐└─┬─┘       │                                                                                                                                                                                                                                                                                                                                                                ┌───┐                                                  ┌───┐                                                                                                                                                                                                                                                                             ┌─┴─┐└─┬─┘┌─┴─┐┌─────────┐                         ┌─┴─┐    └─┬─┘    ┌─┴─┐   ┌─────────┐                                          ┌─┴─┐                                                      ┌─────────┐                                                                                                          ┌─────────┐┌─┴─┐┌─┴─┐  │       ┌───┐└─┬─┘       │  │       │
q_1: ┤1     ├──┼────■──┤ X ├──┼────■────┼─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────■──┤ X ├──■────────────────────────────────────────■──────┤ X ├──────■────────────────────────────────────────────────────────────■──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────■────────■─────┤ X ├──■──┤ X ├┤

In [27]:
# grade your work!
grade_lab1_ex7(PeresGateTranslation, pg)

Submitting your answer. Please wait...
Congratulations 🎉! Your answer is correct and has been submitted.


In [40]:
%set_env QXToken=5f375a49c4b5eb4c003b2bc33a2e96b3b4acb5acc84a44a21e8b793c87402bf1e900c28b4a09b084434b58633ebc764b020c7d2759699cd41242af3a8c8e313c


env: QXToken=5f375a49c4b5eb4c003b2bc33a2e96b3b4acb5acc84a44a21e8b793c87402bf1e900c28b4a09b084434b58633ebc764b020c7d2759699cd41242af3a8c8e313c


In [8]:
from qiskit_ibm_runtime import QiskitRuntimeService

# Initialize the service
service = QiskitRuntimeService()

# Choose the backend
BACKEND = service.backend('ibm_sherbrooke')

# Define the 1-D chain of qubits on ibm_sherbrooke
# Check the backend configuration to ensure qubits are connected in a path-like structure
# Example path for ibm_sherbrooke, modify as needed based on the actual coupling map
PATH_ANSWER = [0, 1, 2, 3, 4]

In [79]:
# Grade the answers
from qc_grader.challenges.qgss_2024 import grade_lab2_ex1
grade_lab2_ex1(PATH_ANSWER, BACKEND)

Submitting your answer. Please wait...
Nice work, all your answers look correct!


In [46]:
qq = QuantumCircuit(5)
qq.h(4)
qq.cx(0,1)
qq.cx(1, 2)
qq.cx(2, 3)
qq.cx(0, 2)
qq.draw()

q_0: ──■──────────────■──
     ┌─┴─┐            │  
q_1: ┤ X ├──■─────────┼──
     └───┘┌─┴─┐     ┌─┴─┐
q_2: ─────┤ X ├──■──┤ X ├
          └───┘┌─┴─┐└───┘
q_3: ──────────┤ X ├─────
     ┌───┐     └───┘     
q_4: ┤ H ├───────────────
     └───┘

In [45]:
BACKEND.basis_gates

['ecr', 'id', 'rz', 'sx', 'x']

In [9]:
BACKEND = service.backend('ibm_sherbrooke')

PATH_ANSWER = [ 0,1, 2, 3, 4, 5]

def make_pairs(path:list):
    for i in range(len(path)-1):
        yield [path[i],path[i+1]]
        
coupling_map = BACKEND.configuration().coupling_map        
        
def make_layers(pair:list[tuple[int,int]]):
    lay_1 = []
    lay_2 = []
    for i , p in enumerate(pair):
        if tuple(p) not in list(BACKEND.coupling_map):
            p = tuple(reversed(p))
        if i % 2 ==0:
            lay_1.append(p)
        else:
            lay_2.append(p)
    return lay_1 , lay_2
LAYER_1_ANSWER , LAYER_2_ANSWER = make_layers(make_pairs(PATH_ANSWER))

In [6]:
# grade part 2
from qc_grader.challenges.qgss_2024 import grade_lab2_ex2
grade_lab2_ex2(LAYER_1_ANSWER, LAYER_2_ANSWER, PATH_ANSWER, BACKEND)

Submitting your answer. Please wait...
Nice work, all your answers look correct!


In [10]:
qq = QuantumCircuit(6)
for i in range(len(LAYER_1_ANSWER)):
    f = LAYER_1_ANSWER[i][0]
    s = LAYER_1_ANSWER[i][1]
    qq.cx(f,s)
for i in range(len(LAYER_2_ANSWER)):
    f = LAYER_2_ANSWER[i][0]
    s = LAYER_2_ANSWER[i][1]
    qq.cx(f,s)    
qq.draw()


┌───┐     
q_0: ┤ X ├─────
     └─┬─┘     
q_1: ──■────■──
     ┌───┐┌─┴─┐
q_2: ┤ X ├┤ X ├
     └─┬─┘├───┤
q_3: ──■──┤ X ├
     ┌───┐└─┬─┘
q_4: ┤ X ├──■──
     └─┬─┘     
q_5: ──■───────

In [11]:
CIRC_1_ANSWER =  QuantumCircuit(BACKEND.num_qubits)
CIRC_2_ANSWER = QuantumCircuit(BACKEND.num_qubits)
for c , l in [(CIRC_1_ANSWER,LAYER_1_ANSWER),(CIRC_2_ANSWER,LAYER_2_ANSWER)]:
    for i , j in l:
        c.ecr(i,j)

GATE_NAME = 'ecr'

In [12]:
# grade part 3
from qc_grader.challenges.qgss_2024 import grade_lab2_ex3
grade_lab2_ex3(CIRC_1_ANSWER, LAYER_1_ANSWER, GATE_NAME, BACKEND)
grade_lab2_ex3(CIRC_2_ANSWER, LAYER_2_ANSWER, GATE_NAME, BACKEND)


Submitting your answer. Please wait...
Nice work, all your answers look correct!
Submitting your answer. Please wait...
Nice work, all your answers look correct!


In [38]:
CIRC_2_ANSWER.draw(fold=-1,idle_wires=False)

┌──────┐
q_1: ┤0     ├
     │  Ecr │
q_2: ┤1     ├
     ├──────┤
q_3: ┤1     ├
     │  Ecr │
q_4: ┤0     ├
     └──────┘

In [13]:
from qiskit.circuit import Parameter
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager

def eplg_circuit(num_qubits: int, depth: int, layer: QuantumCircuit, qubits: tuple[int, ...]) -> QuantumCircuit:
    if depth % 2 != 0:
        raise ValueError(f"The depth must be even, got {depth}")

    def parameters():
        _depth = 0
        while True:
            for zi in range(3):
                for q in range(num_qubits):
                    yield Parameter(f"d{_depth}_q{q}_z{zi}")
            _depth += 1
    _parameters = parameters()

    circ = QuantumCircuit(num_qubits)

    def _1q_layer():
        for j in range(3):
            for k in qubits:
                circ.rz(next(_parameters), k)
            if j == 2:
                continue
            circ.sx(qubits)

    for _depth in range(depth // 2):
        circ.barrier(qubits)
        _1q_layer()
        circ.barrier(qubits)
        circ.compose(layer, inplace=True)
    circ.barrier(qubits)
    _1q_layer()
    circ.barrier(qubits)
    
    # Assuming mirroring is fine
    circ = circ.compose(circ.inverse())

    circ.measure_active()

    pm = generate_preset_pass_manager(basis_gates=["ecr", "sx", "rz"], optimization_level=0)
    circ = pm.run(circ)

    circ.metadata["layer_depth"] = depth
    circ.metadata["qubits"] = qubits
    circ.metadata["num_qubits"] = num_qubits

    return circ

In [14]:
example_circuit_1 = eplg_circuit(
    num_qubits=BACKEND.num_qubits,
    depth=4,
    layer=CIRC_1_ANSWER,
    qubits=PATH_ANSWER,
)

In [15]:
example_circuit_2 = eplg_circuit(
    num_qubits=BACKEND.num_qubits,
    depth=4,
    layer=CIRC_2_ANSWER,
    qubits=PATH_ANSWER,
)

In [16]:
example_circuit_2.draw(fold=-1,idle_wires=False)

░ ┌──────────────┐┌────┐ ┌──────────────┐┌────┐┌───────────────┐ ░          ░ ┌───────────────┐┌────┐┌───────────────┐┌────┐┌───────────────┐ ░          ░ ┌───────────────┐┌────┐┌───────────────┐┌────┐┌───────────────┐ ░  ░ ┌────────────────────┐┌─────────┐┌─────────┐┌────┐┌─────────┐┌─────────┐┌────────────────────┐┌─────────┐┌─────────┐┌────┐┌─────────┐┌─────────┐┌────────────────────┐ ░          ░ ┌────────────────────┐┌─────────┐┌─────────┐┌────┐┌─────────┐┌─────────┐┌────────────────────┐┌─────────┐┌─────────┐┌────┐┌─────────┐┌─────────┐┌────────────────────┐ ░          ░ ┌────────────────────┐┌─────────┐┌─────────┐┌────┐┌─────────┐┌─────────┐┌───────────────────┐ ┌─────────┐┌─────────┐┌────┐┌─────────┐┌─────────┐┌───────────────────┐ ░  ░ ┌─┐               
      q_0: ─░─┤ Rz(d0_q0_z0) ├┤ √X ├─┤ Rz(d0_q6_z0) ├┤ √X ├┤ Rz(d0_q12_z0) ├─░──────────░─┤ Rz(d0_q18_z0) ├┤ √X ├┤ Rz(d0_q24_z0) ├┤ √X ├┤ Rz(d0_q30_z0) ├─░──────────░─┤ Rz(d0_q36_z0) ├┤ √X ├┤ Rz(d0_q42_z0) ├┤ √X ├┤ Rz(d0_q48_z0) ├─░──░─┤ Rz(-1.0*d0_q48_z0) ├┤ Rz(π/2) ├┤ Rz(π/2) ├┤ √X ├┤ Rz(π/2) ├┤ Rz(π/2) ├┤ Rz(-1.0*d0_q42_z0) ├┤ Rz(π/2) ├┤ Rz(π/2) ├┤ √X ├┤ Rz(π/2) ├┤ Rz(π/2) ├┤ Rz(-1.0*d0_q36_z0) ├─░──────────░─┤ Rz(-1.0*d0_q30_z0) ├┤ Rz(π/2) ├┤ Rz(π/2) ├┤ √X ├┤ Rz(π/2) ├┤ Rz(π/2) ├┤ Rz(-1.0*d0_q24_z0) ├┤ Rz(π/2) ├┤ Rz(π/2) ├┤ √X ├┤ Rz(π/2) ├┤ Rz(π/2) ├┤ Rz(-1.0*d0_q18_z0) ├─░──────────░─┤ Rz(-1.0*d0_q12_z0) ├┤ Rz(π/2) ├┤ Rz(π/2) ├┤ √X ├┤ Rz(π/2) ├┤ Rz(π/2) ├┤ Rz(-1.0*d0_q6_z0) ├─┤ Rz(π/2) ├┤ Rz(π/2) ├┤ √X ├┤ Rz(π/2) ├┤ Rz(π/2) ├┤ Rz(-1.0*d0_q0_z0) ├─░──░─┤M├───────────────
            ░ ├──────────────┤├────┤ ├──────────────┤├────┤├───────────────┤ ░ ┌──────┐ ░ ├───────────────┤├────┤├───────────────┤├────┤├───────────────┤ ░ ┌──────┐ ░ ├───────────────┤├────┤├───────────────┤├────┤├───────────────┤ ░  ░ ├────────────────────┤├─────────┤├─────────┤├────┤├─────────┤├─────────┤├────────────────────┤├─────────┤├─────────┤├────┤├─────────┤├─────────┤├────────────────────┤ ░ ┌──────┐ ░ ├────────────────────┤├─────────┤├─────────┤├────┤├─────────┤├─────────┤├────────────────────┤├─────────┤├─────────┤├────┤├─────────┤├─────────┤├────────────────────┤ ░ ┌──────┐ ░ ├────────────────────┤├─────────┤├─────────┤├────┤├─────────┤├─────────┤├───────────────────┤ ├─────────┤├─────────┤├────┤├─────────┤├─────────┤├───────────────────┤ ░  ░ └╥┘┌─┐            
      q_1: ─░─┤ Rz(d0_q1_z0) ├┤ √X ├─┤ Rz(d0_q7_z0) ├┤ √X ├┤ Rz(d0_q13_z0) ├─░─┤0     ├─░─┤ Rz(d0_q19_z0) ├┤ √X ├┤ Rz(d0_q25_z0) ├┤ √X ├┤ Rz(d0_q31_z0) ├─░─┤0     ├─░─┤ Rz(d0_q37_z0) ├┤ √X ├┤ Rz(d0_q43_z0) ├┤ √X ├┤ Rz(d0_q49_z0) ├─░──░─┤ Rz(-1.0*d0_q49_z0) ├┤ Rz(π/2) ├┤ Rz(π/2) ├┤ √X ├┤ Rz(π/2) ├┤ Rz(π/2) ├┤ Rz(-1.0*d0_q43_z0) ├┤ Rz(π/2) ├┤ Rz(π/2) ├┤ √X ├┤ Rz(π/2) ├┤ Rz(π/2) ├┤ Rz(-1.0*d0_q37_z0) ├─░─┤0     ├─░─┤ Rz(-1.0*d0_q31_z0) ├┤ Rz(π/2) ├┤ Rz(π/2) ├┤ √X ├┤ Rz(π/2) ├┤ Rz(π/2) ├┤ Rz(-1.0*d0_q25_z0) ├┤ Rz(π/2) ├┤ Rz(π/2) ├┤ √X ├┤ Rz(π/2) ├┤ Rz(π/2) ├┤ Rz(-1.0*d0_q19_z0) ├─░─┤0     ├─░─┤ Rz(-1.0*d0_q13_z0) ├┤ Rz(π/2) ├┤ Rz(π/2) ├┤ √X ├┤ Rz(π/2) ├┤ Rz(π/2) ├┤ Rz(-1.0*d0_q7_z0) ├─┤ Rz(π/2) ├┤ Rz(π/2) ├┤ √X ├┤ Rz(π/2) ├┤ Rz(π/2) ├┤ Rz(-1.0*d0_q1_z0) ├─░──░──╫─┤M├────────────
            ░ ├──────────────┤├────┤ ├──────────────┤├────┤├───────────────┤ ░ │  Ecr │ ░ ├───────────────┤├────┤├───────────────┤├────┤├───────────────┤ ░ │  Ecr │ ░ ├───────────────┤├────┤├───────────────┤├────┤├───────────────┤ ░  ░ ├────────────────────┤├─────────┤├─────────┤├────┤├─────────┤├─────────┤├────────────────────┤├─────────┤├─────────┤├────┤├─────────┤├─────────┤├────────────────────┤ ░ │  Ecr │ ░ ├────────────────────┤├─────────┤├─────────┤├────┤├─────────┤├─────────┤├────────────────────┤├─────────┤├─────────┤├────┤├─────────┤├─────────┤├────────────────────┤ ░ │  Ecr │ ░ ├────────────────────┤├─────────┤├─────────┤├────┤├─────────┤├─────────┤├───────────────────┤ ├─────────┤├─────────┤├────┤├─────────┤├─────────┤├───────────────────┤ ░  ░  ║ └╥┘┌─┐         
      q_2: ─░─┤ Rz(d0_q2_z0) ├┤ √X ├─┤ Rz(d0_q8_z0) ├┤ √X ├┤ Rz(d0_q14_z0) ├─░─┤1 

In [17]:
import numpy as np

from qiskit.primitives.containers.bindings_array import BindingsArray

def get_clifford_rz_samples(
    circ: QuantumCircuit,
    num_samples: int,
    rng: np.random.Generator,
) -> BindingsArray:
    """Uniformly sample the Rz parameters in a `QuantumCircuit`
    from -pi, -pi/2, 0, +pi/2, +pi.
    """

    _allowed_ops = {'rz', 'sx', 'sxdg', 'ecr', 'barrier', 'measure'}
    if (_other := set(circ.count_ops().keys()) - _allowed_ops):
        raise ValueError(f"Circuit must only contain ops: {_allowed_ops}, got {_other}")

    sampled_pars = rng.integers(-2, 3, size=(num_samples, len(circ.parameters))) * (np.pi/2)

    return BindingsArray({
        tuple(circ.parameters): sampled_pars
    })

In [18]:
num_samples = 10
ARRAY_1_ANSWER = get_clifford_rz_samples(
    circ=example_circuit_1,
    num_samples=num_samples,
    rng=np.random.default_rng(42))

In [19]:
# grade part 4
from qc_grader.challenges.qgss_2024 import grade_lab2_ex4
grade_lab2_ex4(ARRAY_1_ANSWER, example_circuit_1, num_samples)

Submitting your answer. Please wait...
Nice work, all your answers look correct! 
 Notice that the shape of the BindingsArray is [10], and there are {num_parameters} parameters.


In [41]:
from qiskit.circuit.library import EfficientSU2

num_qubits = 50
reps = 2
abstract_circuit = EfficientSU2(num_qubits, reps=reps, entanglement="pairwise")

In [42]:
import numpy as np

num_parameters = abstract_circuit.num_parameters
param_values = np.random.uniform(-np.pi, np.pi, size=num_parameters)

abstract_circuit.assign_parameters(param_values, inplace=True)

In [43]:
from qiskit.circuit.library import UnitaryOverlap

abstract_circuit.barrier()
abstract_circuit = UnitaryOverlap(abstract_circuit, abstract_circuit)

In [44]:
from qiskit.quantum_info import SparsePauliOp

paulis = ["".join("Z" if i == q else "I" for i in range(num_qubits)) for q in range(num_qubits)]
abstract_observables = [SparsePauliOp(pauli) for pauli in paulis]

In [45]:
from qiskit_ibm_runtime import QiskitRuntimeService

service = QiskitRuntimeService()
backend = service.least_busy(min_num_qubits=127)
backend.name

'ibm_osaka'

In [46]:
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager

pm = generate_preset_pass_manager(backend=backend, optimization_level=3)
target_circuit = pm.run(abstract_circuit)

In [47]:
layout = target_circuit.layout
target_observables = [abs_obs.apply_layout(layout=layout) for abs_obs in abstract_observables]

In [50]:
from qiskit_ibm_runtime import EstimatorV2 as Estimator, EstimatorOptions, Batch

# Define the primitive unified bloc (PUB) for Estimator jobs
# More on PUB: https://docs.quantum.ibm.com/api/qiskit/primitives
pub = (target_circuit, target_observables)
default_shots = 10_000

# list for saving job results
primitive_results = []

In [51]:
# Submit Exercise 1 to 3 inside a Batch execution mode
with Batch(backend=backend) as batch:
    # Excercise-1: No mitigation (worked out for you)
    ## https://docs.quantum.ibm.com/run/configure-error-mitigation#turn-off-all-error-mitigation-and-error-suppression
    options_ex1 = EstimatorOptions() # some suppression and mitigation are enabled by default
    options_ex1.optimization_level = 0
    options_ex1.resilience_level = 0
    options_ex1.default_shots = default_shots
    
    # Instantiate `Estimator` with options
    estimator = Estimator(options=options_ex1)
    # Submit job
    job_ex1 = estimator.run(pubs=[pub])

    
    # Excercise-2: Dynamical Decoupling (DD)
    from qiskit_ibm_runtime import SamplerOptions, EstimatorOptions
    options_ex2 = EstimatorOptions(optimization_level=0, resilience_level=0)
    options_ex2.default_shots = default_shots
    options_ex2.dynamical_decoupling.enable = True
    options_ex2.dynamical_decoupling.sequence_type = 'XX'
    options_ex2.dynamical_decoupling.extra_slack_distribution = 'middle'
    options_ex2.dynamical_decoupling.scheduling_method = 'alap'

    
    estimator = Estimator(options=options_ex2)
    job_ex2 = estimator.run(pubs=[pub])

    
    # Excercise-3: Measurement mitigation (TREX)
    from qiskit_ibm_runtime import EstimatorOptions
    options_ex3 = EstimatorOptions(optimization_level=0, resilience_level=0)
    options_ex3.default_shots = default_shots
    options_ex3.resilience.measure_mitigation = True 
    options_ex3.resilience.measure_noise_learning.num_randomizations = 32
    options_ex3.resilience.measure_noise_learning.shots_per_randomization = 'auto'
    options_ex3.twirling.enable_measure = True

    
    estimator = Estimator(options=options_ex3)
    job_ex3 = estimator.run(pubs=[pub])

# Wait for first 3 jobs to complete. Fetch results when done
primitive_results.append(job_ex1.result())
primitive_results.append(job_ex2.result())
primitive_results.append(job_ex3.result())

KeyboardInterrupt: 

In [29]:
# Submit Exercise 4a, 4b, and 5 inside another Batch execution mode
with Batch(backend=backend) as batch:
    # Excercise-4a: Zero Noise Extrapolation (extrapolator="exponential" | noise_factors=(1, 3, 5))
    from qiskit_ibm_runtime import SamplerOptions, EstimatorOptions
    options_ex4a = EstimatorOptions(optimization_level=0, resilience_level=0)
    options_ex4a.default_shots = default_shots
    options_ex4a.resilience.zne_mitigation = True
    options_ex4a.resilience.zne.noise_factors = (1,3,5)
    options_ex4a.resilience.zne.extrapolator = ('exponential')

    estimator = Estimator(options=options_ex4a)
    job_ex4a = estimator.run(pubs=[pub])

    
    # Excercise-4b: Zero Noise Extrapolation (use: extrapolator="linear" and noise_factors=(1, 3, 5))
    options_ex4b = EstimatorOptions()

    options_ex4b = EstimatorOptions(optimization_level=0, resilience_level=0)
    options_ex4b.default_shots = default_shots
    options_ex4b.resilience.zne_mitigation = True
    options_ex4b.resilience.zne.noise_factors = (1,3,5)
    options_ex4b.resilience.zne.extrapolator = ('linear')
    
    estimator = Estimator(options=options_ex4b)
    job_ex4b = estimator.run(pubs=[pub])

    
    # Excercise-5: Gate Twirling + Zero Noise Extrapolation (use: extrapolator=("exponential", "linear") and noise_factors=(1, 3, 5))

    options_ex5 = EstimatorOptions(optimization_level=0, resilience_level=0)
    
    options_ex5.default_shots = default_shots
    
    options_ex5.twirling.enable_gates = True
    options_ex5.twirling.enable_measure = False
    options_ex5.twirling.num_randomizations = 'auto'
    options_ex5.twirling.shots_per_randomization = 'auto'
    options_ex5.twirling.strategy = 'active-accum'
    
    
    options_ex5.resilience.zne_mitigation = True
    options_ex5.resilience.zne.noise_factors = (1,3,5)
    options_ex5.resilience.zne.extrapolator = ('exponential' ,'linear')
    
    
    estimator = Estimator(options=options_ex5)
    job_ex5 = estimator.run(pubs=[pub])

# Wait for next 3 jobs to complete. Fetch results when done
primitive_results.append(job_ex4a.result())
primitive_results.append(job_ex4b.result())
primitive_results.append(job_ex5.result())

KeyboardInterrupt: 

In [ ]:
# Submit Exercise 6 in Job execution mode as it is a single job
# Excercise-6: All
from qiskit_ibm_runtime import QiskitRuntimeService

options_ex6 = EstimatorOptions(optimization_level=0, resilience_level=0)

options_ex6.default_shots = default_shots

options_ex6.dynamical_decoupling.enable = True
options_ex6.dynamical_decoupling.sequence_type = 'XX'
options_ex6.dynamical_decoupling.extra_slack_distribution = 'middle'
options_ex6.dynamical_decoupling.scheduling_method = 'alap'

options_ex6.twirling.enable_gates = True
options_ex6.twirling.enable_measure = True
options_ex6.twirling.num_randomizations = 'auto'
options_ex6.twirling.shots_per_randomization = 'auto'
options_ex6.twirling.strategy = 'active-accum'

options_ex6.resilience.measure_mitigation = True 
options_ex6.resilience.measure_noise_learning.num_randomizations = 32
options_ex6.resilience.measure_noise_learning.shots_per_randomization = 'auto'

options_ex6.resilience.zne_mitigation = True
options_ex6.resilience.zne.noise_factors = (1,3,5)
options_ex6.resilience.zne.extrapolator = ('exponential' ,'linear')    


# Note: explicitly set `mode=backend` in Job execution mode.
# Inside Batch context manager, `Estimator` knows the context/backend implicitly
# However, without the context manager, we must set `mode` explicitly
estimator = Estimator(mode=backend, options=options_ex6)
job_ex6 = estimator.run(pubs=[pub])
primitive_results.append(job_ex6.result())